### 进程篇

#### 进程的两种调用形式

In [3]:
#方式一：直接调用
import multiprocessing as mp
import time
from func import say_hi  #由于Ipython的限制，只能调用

# def say_hi(words):
#     time.sleep(2)
#     print("I say:{}".format(words))
#     time.sleep(2)

if __name__=='__main__':
    process_1=mp.Process(target=say_hi,args=("hello world!",),name="Process_1")
    process_2=mp.Process(target=say_hi,args=("Good!",),name="Process_2")

    process_1.start()
    process_2.start()

    process_1.join()
    process_2.join()

    print("Ending...")

Ending...


In [7]:
#继承类
import multiprocessing as mp
import time
from func import MyProcess

# class MyProcess(mp.Process):
#     def __init__(self,name):
#         super().__init__()
#         self._name=name
#     def run(self):
#         print("%s is running"%self._name)
#         time.sleep(2)
#         print("%s is done"%self._name)
        
if __name__=='__main__':
    process_1=MyProcess("hello world!")
    process_2=MyProcess("Good!")
    
    process_1.start()
    process_2.start()
    
    process_1.join()
    process_2.join()
    
    print("Ending...")
    

Ending...


#### 查看进程的pid与ppid

使用`pid`和`ppid`可以分别查看子进程和父进程的进程ID。

In [2]:
import multiprocessing as mp
import time,os
from func import task

# def task():
#     print("%s is running,parent id is <%s>"%(os.getpid(),os.getppid()))
#     time.sleep(2)
#     print("ts is done,parent id is <%s>"%(os.getpid(),os.getppid()))

if __name__=='__main__':
    p=mp.Process(target=task)
    p.start()
    
    print("主",os.getpid(),os.getppid())

主 18179 17043


#### 进程之间的内存空间是隔离的

**与线程不同，进程没有任何共享状态，进程修改的数据，改动仅限于该进程内。**

In [8]:
from multiprocessing import process

n=100

def work():
    global n
    n=0
    print("子进程内:",n)  #打印:0

if __name__=='__main__':
    p=Process(target=work,)
    p.start()
    p.join()
    print("主进程内:",n) #打印100

### 队列的使用 

multiprocessing模块提供了IPC进程之间的通信，队列以及管道。

Queue([maxsize]):创建共享的进程队列，Queue是多进程安全的队列，可以使用Queue实现多进程之间的数据传递。

maxsize是队列中允许最大项数，可以放置任意类型的数据，省略则无大小限制。

主要方法:

　　* `q.put()`方法用以插入数据到队列中。数据不宜过大。

　　* `q.get()`方法可以从队列读取并且删除一个元素。

In [3]:
from multiprocessing import Process,Queue
 
q=Queue(3)
 
#put ,get ,put_nowait,get_nowait,full,empty
q.put(1)
q.put(2)
q.put(3)
print(q.full())  # 判断是否满了
# q.put(4) #再放就阻塞住了
 
print(q.get())
print(q.get())
print(q.get())
print(q.empty())  # 判断是否空了
# print(q.get()) #再取就阻塞住了

True
1
2
3
True


In [ ]:
#使用队列实现生产者消费者模型
from multiprocessing import Process, Queue
import time
 
def producer(q, name):
    for i in range(3):
        res = '包子%s' % i
        time.sleep(0.5)
        print('%s 生产了%s' % (name, res))
 
        q.put(res)
 
def consumer(q, name):
    while True:
        res = q.get()
        if res is None: break
        time.sleep(1)
        print('%s 吃了%s' % (name, res))
 
 
if __name__ == '__main__':
    # 容器
    q = Queue()
 
    # 生产者们
    p1 = Process(target=producer, args=(q, '生产者1'))
    p2 = Process(target=producer, args=(q, '生产者2'))
    p3 = Process(target=producer, args=(q, '生产者3'))
 
    # 消费者们
    c1 = Process(target=consumer, args=(q, '消费者1'))
    c2 = Process(target=consumer, args=(q, '消费者2'))
 
    p1.start()
    p2.start()
    p3.start()
    c1.start()
    c2.start()
 
    p1.join()
    p2.join()
    p3.join()
    q.put(None)
    q.put(None)
    print('主')

### 共享内存

共享内存也可以实现进程间的通信。定义共享内存中的数据可以被不同进程读取和修改。

| Type code | C Type             | Python Type       | Minimum size in bytes |
| --------- | ------------------ | ----------------- | --------------------- |
| `'b'`     | signed char        | int               | 1                     |
| `'B'`     | unsigned char      | int               | 1                     |
| `'u'`     | Py_UNICODE         | Unicode character | 2                     |
| `'h'`     | signed short       | int               | 2                     |
| `'H'`     | unsigned short     | int               | 2                     |
| `'i'`     | signed int         | int               | 2                     |
| `'I'`     | unsigned int       | int               | 2                     |
| `'l'`     | signed long        | int               | 4                     |
| `'L'`     | unsigned long      | int               | 4                     |
| `'q'`     | signed long long   | int               | 8                     |
| `'Q'`     | unsigned long long | int               | 8                     |
| `'f'`     | float              | float             | 4                     |
| `'d'`     | double             | float             | 8                     |

In [4]:
import multiprocessing as mp

#通过使用Value将数据存储在一个共享的内存表中
#其中d和i参数用来设置数据类型
#d表示一个双精度浮点类型,i表示一个带符号的整型
value1=mp.Value('i',0)
value2=mp.Value('d',3.14)

#这里的Array只能是一维的，不能是多维的，且需要定义数据形式
array=mp.Array('i',[1,2,3,4])

### 进程池Pool

进程池就是我们将所有运行的东西，放到池子里，python会自行解决多进程的问题。

1. pool默认调动是CPU的核数,Pool调用的函数有返回值，而Process调用的函数则没有返回值
2. 对Pool对象调用join()方法会等待所有子进程执行完毕，调用join()之前必须先调用close(),调用cloes()之后就不能继续添加新的Process
3. map(func,seq)放入迭代参数，返回多个结果
4. apply_async(func,parm)只能执行一次

In [6]:
from multiprocessing import Pool
import os,time,random

def long_time_test(name):
    print('Run task %s (%s)...'%(name,os.getpid()))
    start=time.time()
    time.sleep(random.random()*3)
    end=time.time()
    print('Task %s runs %0.2f seconds.'%(name,(end-start)))

if __name__=='__main__':
    print('Parent process %s.'%os.getpid())
    p=Pool(4)
    for i in range(5):
        p.apply_async(long_time_test,args=(i,))
    print('Waiting for all subprocess done...')
    p.close()
    p.join()
    print('All subprocess done.')

Parent process 1863.
Run task 1 (2098)...
Run task 0 (2097)...
Run task 3 (2100)...
Run task 2 (2099)...
Waiting for all subprocess done...
Task 3 runs 0.29 seconds.
Run task 4 (2100)...
Task 2 runs 1.75 seconds.
Task 0 runs 2.59 seconds.
Task 1 runs 2.72 seconds.
Task 4 runs 2.93 seconds.
All subprocess done.


### 进程锁Lock

In [14]:
#不加进程锁，两个进程争夺共享内存v
import multiprocessing as mp
import time,os

def job(v,num):
    for _ in range(5):
        time.sleep(num)
        v.value+=num   #v.value获取共享变量值
        print('curr process',os.getpid(),v.value)

def multicore():
    v=mp.Value('i',0)
    p1=mp.Process(target=job,args=(v,1))
    p2=mp.Process(target=job,args=(v,3))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__=='__main__':
    multicore()

curr process 2749 1
curr process 2749 2
curr process 2750 5
curr process 2749 6
curr process 2749 7
curr process 2749 8
curr process 2750 11
curr process 2750 14
curr process 2750 17
curr process 2750 20


In [17]:
#使用进程锁，保证运行时一个进程对锁内内容的独占
#使用l.acquire()加锁,l.release()释放锁
import multiprocessing as mp
import time

def job(v,num,l):
    l.acquire()
    for _ in range(5):
        time.sleep(num)
        v.value+=num
        print(v.value)
    l.release()

def multicore():
    l=mp.Lock()
    v=mp.Value('i',0)
    p1=mp.Process(target=job,args=(v,3,l))
    p2=mp.Process(target=job,args=(v,1,l))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__=='__main__':
    multicore()

3
6
9
12
15
16
17
18
19
20


#### 后话

##### 多线程与多进程的区别

1. 开启速度
    * 线程的创建开销极小
    * 创建进程，操作系统要申请内存空间，拷贝父进程地址空间到子进程，开销远大于线程
2. pid的不同
    * 在主进程下开启多个线程,每个线程都跟主进程的pid一样
    * 开多个进程,每个进程都有不同的pid
    * 同一进程内的多个线程共享该进程的地址空间，父进程与子进程不共享地址空间, 进程之间的地址空间是隔离的

3. 多线程适用于IO密集型任务，多进程适用于计算密集型任务。

##### 应用：

* 多线程用于IO密集型，如socket，爬虫，web
* 多进程用于计算密集型，如金融分析